In [36]:
import os
import re
import json
import pandas as pd
from src.utils import read_config
from prompts import gemini_prompts
import google.generativeai as genai
import google.ai.generativelanguage as glm

from langchain_huggingface import HuggingFaceEmbeddings
from langchain.document_loaders import DataFrameLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores.faiss import FAISS, DistanceStrategy
from langchain.chains import RetrievalQA
from langchain.chains import ConversationalRetrievalChain
from langchain.prompts import PromptTemplate
from langchain_google_genai import (
    ChatGoogleGenerativeAI,
    HarmBlockThreshold,
    HarmCategory,
)

from langchain.chains.query_constructor.base import AttributeInfo
from langchain.retrievers.self_query.base import SelfQueryRetriever

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

## 0609 notes
1. Add KG(knowledge graph) as part of metadata
2. Use SelfQueryRetriver to get the top_k answer
3. Override the `_get_docs_with_query` to use the `similarity_search_with_score` method(or with `similarity_search_with_score_by_vector`)

In [2]:
organized_result = pd.read_csv('temp_result/organized_result_1.csv')
organized_result.shape

(76, 5)

In [32]:
organized_result.columns

Index(['document', 'target', 'reason', 'fact', 'metadata'], dtype='object')

In [3]:
configs = read_config('.env/configs.json')
api_key =configs['g_key']
os.environ["GOOGLE_API_KEY"] = api_key
genai.configure(api_key=api_key)

In [4]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1000,
    chunk_overlap = 500,
    length_function= len
)

In [5]:
reason_loader = DataFrameLoader(organized_result, page_content_column='reason')
reason_data = reason_loader.load()

target_loader = DataFrameLoader(organized_result, page_content_column='target')
target_data = target_loader.load()

reason_documents = text_splitter.transform_documents(reason_data)
print("已為所有文件進行 chunk ",len(reason_documents),"筆")

已為所有文件進行 chunk  76 筆


In [35]:
reason_data[0].metadata

{'document': '34829.pdf',
 'target': '行政院原住民族委員會及花蓮縣政府',
 'fact': '據立法委員函轉民眾陳訴行政院原住民族委員會(下稱原民會)補助花蓮區漁會給二予漁民架設衛星導航系統疑有弊端，惟該會遲未盡查處之責，涉有違失等情經本院調查結果，行政院原住民族委員會及花蓮縣政府辦理補助花蓮區漁會採購「三機一體彩色漁探衛星導航電子海圖儀及船用ＤＳＢ無線電對講機」乙案，確有下列違失一、花蓮區漁會辦理「三機一體彩色漁探衛星導航電子海圖儀及船用ＤＳＢ無線電對講機採購案」，未能確實依政府採購法之規定辦理，採購作業錯誤叢生，補助機關行政院原住民族委員會及花蓮縣政府原住民行政局顯然疏於監督，核有未當，應予檢討改進(一)查行政院原住民族委員會民國(下同)九十二年度補助花蓮縣政府支付花蓮區漁會辦理部落產業發展計畫｜東海岸原住民漁業輔導計畫經費新台幣(下同)一七七萬元，並納入該府預算執行，其中花蓮區漁會辦理「三機一體彩色漁探衛星導航電子海圖儀及船用ＤＳＢ無線電對講機採購案」，於九十二年十月十四日開標，由友漁企業公司以一二五萬元得標，惟經查該漁會辦理該採購案違反政府採購法諸多規定，核有下列缺失１、該漁會收受三家廠商投標文件，未於標封袋登記廠商送達方式與時間，亦無簽收紀錄可稽；審查廠商投標作業，僅於證件封袋上予以勾選，無審查之人員簽名負責；又未能提供廠商繳納押標金之紀錄憑核，對於其繳交之額度、支票號碼及領回日期等資料，均無從查核２、該採購案並非特殊或鉅額採購，卻限定投標資格為資本額新台幣五百萬元以上之廠商，上開廠商須具有相當財力之規定，核與政府採購法第三十六條第二項暨「投三標廠商資格與特殊或巨額採購認定標準」第五條第一項第三款規定不符核有非屬特殊或鉅額採購，卻限定財力之特定資格，不當限制廠商競爭之情事，顯已違反採購法第三十七條「機關訂定投標廠商之資格，不得不當限制競爭，並以確認廠商具備履行契約所必須之能力者為限」之規定３、該採購案招標公告刊登標的名稱為「漁船衛星定位儀及通訊系統工程採購」及標的分類為「漁業工程」，惟查採購合約名稱為「三機一體彩色漁探衛星導航電子海圖儀及船用ＤＳＢ無線電對講機」，且實際辦理內容則為設備類之財物採購，核有採購屬性刊登錯誤４、該採購案未得標廠商陸青科技股份有限公司，其營利事業登記證、經濟部公司執照及交通部電信管制器材經營

In [6]:
model_name = "sentence-transformers/distiluse-base-multilingual-cased-v1"
embeddings = HuggingFaceEmbeddings(model_name=model_name)

e:\miniconda3\envs\project_envelop\lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


### Function for getting triple

In [7]:
llm = ChatGoogleGenerativeAI(
    model="gemini-pro",
    convert_system_message_to_human=True,
    safety_settings={
        HarmCategory.HARM_CATEGORY_HATE_SPEECH: HarmBlockThreshold.BLOCK_NONE,
        HarmCategory.HARM_CATEGORY_HARASSMENT: HarmBlockThreshold.BLOCK_NONE,
        HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: HarmBlockThreshold.BLOCK_NONE,
    },
)

In [8]:
test_data = reason_documents[:3]

In [18]:
test_dict = {}
for i, page in enumerate(reason_documents):
    

    triple_prompt = PromptTemplate.from_template(gemini_prompts.TRIPLE_PROMPT)
    chain = triple_prompt | llm
    triple_data = {
        "reference": page.page_content
    }

    triple_response = chain.invoke(triple_data)

    triple_response_result = re.sub(r'[```json\n]', '', triple_response.content).replace("實體", "entity").replace("關係", "relationship").split("、")[0]

    to_json_prompt = PromptTemplate.from_template(gemini_prompts.TO_JSON_PROMPT)
    chain = to_json_prompt | llm
    data = {
        "reference": triple_response_result
    }

    to_json_response = chain.invoke(data)
    
    triple_json_result = re.sub(r'[\n]','',to_json_response.content)
    triple_json_result = json.loads(f'[{triple_json_result}]')
    
    test_dict[i] = triple_json_result

e:\miniconda3\envs\project_envelop\lib\site-packages\langchain_google_genai\chat_models.py:344: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")
e:\miniconda3\envs\project_envelop\lib\site-packages\langchain_google_genai\chat_models.py:344: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")
e:\miniconda3\envs\project_envelop\lib\site-packages\langchain_google_genai\chat_models.py:344: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")
e:\miniconda3\envs\project_envelop\lib\site-packages\langchain_google_genai\chat_models.py:344: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")
e:\miniconda3\envs\project_envelop\lib\site-packages\langchain_google_ge

In [19]:
test_dict

{0: [{'entity1': '花蓮區漁會',
   'relationship': '辦理',
   'entity2': '「三機一體彩色漁探衛星導航電子海圖儀及船用ＤＳＢ無線電對講機採購案」'},
  {'entity1': '花蓮縣政府', 'relationship': '疏於監督', 'entity2': '花蓮區漁會'},
  {'entity1': '行政院原住民族委員會', 'relationship': '遲延查復', 'entity2': '立法委員'}],
 1: [{'entity1': '行政院衛生署', 'relationship': '辦理', 'entity2': '九十至九十三年度補'}],
 2: [{'entity1': '台北市政府', 'relationship': '未善盡土地使用管制'}]}

In [8]:
vector_db_path = "./vector_db/"

if not os.path.exists(vector_db_path):
    os.makedirs(vector_db_path)
    print(f"Created folder: {vector_db_path}")
    
vector_store = FAISS.from_documents(reason_documents, embeddings, normalize_L2=True, distance_strategy=DistanceStrategy.MAX_INNER_PRODUCT)
vector_store.save_local(folder_path=vector_db_path)

# load FAISS
vectorstore = FAISS.load_local(folder_path=vector_db_path, allow_dangerous_deserialization=True, embeddings=embeddings)

In [9]:
query = "行政院原住民委員會的相關資訊"

embedding_vector = embeddings.embed_query(query)
docs = vectorstore.similarity_search_with_score_by_vector(embedding_vector, k=1)
docs

[(Document(page_content='行政院客家委員會、行政院原住民族委員會及僑務委員會將客家電視、原住民電視及臺灣宏觀電視等頻道節目之製播，依政府採購法委由財團法人公共電視文化事業基金會辦理，雖非無據，惟造成公視基金會與客委會、原民會及僑委會針對上開頻道節目之製播各有立場、互生歧見，致紛爭不斷，詎行政院新聞局坐視上開爭端，非但未積極研修法令，亦未尋求妥適解決辦法，核有怠失，爰依法提案糾正', metadata={'document': '52587.pdf', 'target': '行政院新聞局', 'fact': '行政院客家委員會、行政院原住民族委員會及僑務委員會(下稱客委會、原民會及僑委會)將客家電視、原住民電視及臺灣宏觀電視(下稱客視、原視及宏觀電視)等頻道節目之製播，依政府採購法委由財團法人公共電視文化事業基金會(下稱公視基金會)辦理，雖非無據，惟造成公視基金會與客委會、原民會及僑委會針對上開頻道節目之製播各有立場、互生歧見，致紛爭不斷，詎行政院新聞局(下稱新聞局)坐視上開爭端，非但未積極研修法令，亦未尋求妥適解決辦法，核有怠失，茲提出事實與理由於后一、據陳訴人指陳略以，95年無線公股處理條例於立法院三讀通過，目的在使黨、政、軍退出媒體，95年7月17日行政院召開「研商『無電視事業公股處理條例』規範原民台、客家台及宏觀頻道交由公共電視台製播2相關機關間溝通協調機制之建立問題」會議，會中決議「其製播所需經費之移撥(無政府採購法之適用問題)…」詎客委會、原民會及僑委會未依該決議辦理，仍依政府採購法，以標案形式由公視基金會辦理，實非合理又公視基金會辦理「原、客、宏觀頻道之計畫與製播」，並無獨立之經費得以辦理，而係由政府部門辦理審查、驗收，始得請領款項之情況，違反無線公股處理條例黨政軍退出媒體之精神，造成公共電視新聞自主性、專業性之箝制，詎該公股處理條例對「經費」疏漏等問題，迄未立法補正，亦造成公共電視台(下稱公視)財務經營之困境目前公視代為製播「原、客、宏觀頻道」，亦遭客委會、原民會及僑委會等延宕付款，更造成公視財務經營困難等語云云二、依據「無線電視事業公股處理條例」第14條第3項所定「政府編列預算招標採購或設置之客家電視、原住民電視、台灣宏觀電視等頻道節目之製播，應於本條例公布施行後之次年度起，『交由』公視基金會辦理」目前公視基金會係依照政府採購

In [11]:
response_data = {}
for i, page in enumerate(docs):
    # match_content_list.append(page.page_content)
    # reference[i] = {
    #     "reason": page.page_content,
    #     "metadata": page.metadata
    # }

    summary_prompt = PromptTemplate.from_template(gemini_prompts.SUMMARY_PROMPT)
    chain = summary_prompt | llm
    summary_data = {
        "reference": page[0].page_content
    }

    summary_response = chain.invoke(summary_data)
    
    pattern = r'\*\*摘要：\*\*'
    if re.search(pattern, summary_response.content):
        re.sub(pattern, '', summary_response.content)
        
    # triple_prompt = PromptTemplate.from_template(gemini_prompts.triple_PROMPT)
    # chain = triple_prompt | llm
    # triple_data = {
    #     "reference": page[0].page_content
    # }

    # triple_response = chain.invoke(triple_data)

    # triple_response_result = re.sub(r'[```json\n]', '', triple_response.content).replace("實體", "entity").replace("關係", "relationship").split("、")[0]

    # to_json_prompt = PromptTemplate.from_template(gemini_prompts.TO_JSON_PROMPT)
    # chain = to_json_prompt | llm
    # data = {
    #     "reference": triple_response_result
    # }

    # to_json_response = chain.invoke(data)
    
    # triple_json_result = re.sub(r'[\n]','',to_json_response.content)
    # triple_json_result = json.loads(f'[{triple_json_result}]')

    response_data[i] = {
        "summary": summary_response.content,
        "source": page[0].metadata['document'],
        "score": f"{page[1]:.2f}",
        "target": page[0].metadata['target'],
        "triple": triple_json_result
    }
    
# print("\n資料來源:", response_data[0]['metadata']['document'])

e:\miniconda3\envs\project_envelop\lib\site-packages\langchain_google_genai\chat_models.py:344: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


AttributeError: module 'prompts.gemini_prompts' has no attribute 'triple_PROMPT'

In [21]:
response_data

{0: {'summary': '監察院糾正行政院新聞局，原因如下：\n\n1. 客家、原住民和宏觀電視頻道的節目製作由公視基金會承辦，但公視基金會與客委會、原民會和僑委會對節目製播立場不同，導致糾紛不斷。\n\n2. 行政院新聞局未積極修法或尋求解決方案，任由爭端持續。',
  'source': '52587.pdf',
  'score': '0.43',
  'target': '行政院新聞局',
  'tuple': [{'entity1': '行政院客家委員會',
    'relationship': '委託',
    'entity2': '財團法人公共電視文化事業基金會'},
   {'entity1': '行政院原住民族委員會',
    'relationship': '委託',
    'entity2': '財團法人公共電視文化事業基金會'},
   {'entity1': '僑務委員會', 'relationship': '委託', 'entity2': '財團法人公共電視文化事業基金會'}]}}

In [ ]:
kginfo = [
    AttributeInfo(
        name="triple_result",
        description="triple of the document in json format. It describe the relation between entities",
        type="string"
    )
]